In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers


In [2]:
dataset_path = "."


In [3]:
image_size = (224, 224)
batch_size = 32


In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)


Found 64362 files belonging to 52 classes.
Using 51490 files for training.
Found 64362 files belonging to 52 classes.
Using 12872 files for validation.


In [5]:
class_names = train_ds.class_names
num_classes = len(class_names)

print(num_classes)


52


In [6]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.map(lambda x, y: (x / 255.0, y))
val_ds   = val_ds.map(lambda x, y: (x / 255.0, y))

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds   = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [7]:
model = tf.keras.Sequential([
    layers.Input(shape=(224, 224, 3)),
    layers.Conv2D(32, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation="softmax")
])


In [8]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [9]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Epoch 1/10


1610/1610 [==============================] - 478s 271ms/step - loss: 1.6224 - accuracy: 0.5320 - val_loss: 0.7115 - val_accuracy: 0.7797
Epoch 2/10
1610/1610 [==============================] - 426s 264ms/step - loss: 0.9267 - accuracy: 0.7100 - val_loss: 0.4969 - val_accuracy: 0.8424
Epoch 3/10
1610/1610 [==============================] - 389s 242ms/step - loss: 0.7157 - accuracy: 0.7707 - val_loss: 0.4371 - val_accuracy: 0.8619
Epoch 4/10
1610/1610 [==============================] - 389s 242ms/step - loss: 0.6043 - accuracy: 0.8029 - val_loss: 0.3842 - val_accuracy: 0.8748
Epoch 5/10
1610/1610 [==============================] - 388s 241ms/step - loss: 0.5211 - accuracy: 0.8272 - val_loss: 0.5223 - val_accuracy: 0.8305
Epoch 6/10
1610/1610 [==============================] - 388s 241ms/step - loss: 0.4639 - accuracy: 0.8452 - val_loss: 0.4156 - val_accuracy: 0.8665
Epoch 7/10
1610/1610 [==============================] - 387s 240ms/step - loss: 0.4081 - accuracy: 0.8635 - va

In [10]:
final_train_acc = history.history["accuracy"][-1]
final_val_acc = history.history["val_accuracy"][-1]

print("Final Training Accuracy:", final_train_acc)
print("Final Validation Accuracy:", final_val_acc)


Final Training Accuracy: 0.8995338678359985
Final Validation Accuracy: 0.9057644605636597


In [11]:
model.save("crop_disease_cnn_model.keras")


In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model("crop_disease_mobilenet.keras")



OSError: No file or directory found at C:\Users\rupde\Downloads\VS\models\crop_disease_mobilenet.keras

In [21]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = "test_leaf.jpg"

img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = img_array / 255.0
img_array = np.expand_dims(img_array, axis=0)

predictions = model.predict(img_array)
predicted_class = class_names[np.argmax(predictions)]

print("Predicted Disease:", predicted_class)


1/1 [==============================] - 0s 333ms/step
Predicted Disease: Banana Bract Mosaic Virus Disease


In [23]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False  # freeze pretrained layers

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation="softmax")
])


In [24]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)


In [ ]:
model.save("crop_disease_mobilenet.keras")
